In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp switch_search

In [ ]:
#| export
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix
import anndata as ad
import seaborn as sns
import requests
import sys
import patchworklib as pw
import urllib
import os
import gzip
import shutil
import torch
import matplotlib.pyplot as plt
from scipy.special import gammaln
from scipy.optimize import minimize
from itertools import combinations
from typing import List, Tuple, Dict, Union, Optional
from functools import lru_cache
import pyranges as pr
from pyfaidx import Fasta
import logging
from scipy.stats import chi2
from statsmodels.stats.multitest import multipletests

from allos.anndata_iso import AnnDataIso

In [ ]:
# | export
class SwitchSearch(AnnDataIso):
    def __perform_mle(self, data, total_counts):
        """
        SciPy-based Dirichlet-Multinomial MLE.

        data: (M x K) torch.Tensor of counts
        total_counts: (M,) torch.Tensor of row sums
        Returns: (nll, alpha_hat)
        """
        data_np = data.detach().cpu().numpy()        # shape (M, K)
        total_counts_np = total_counts.detach().cpu().numpy()  # shape (M,)

        # We don't strictly need total_counts_np for the objective,
        # since it's implicit in row sums of data_np.

        K = data_np.shape[1]

        def objective(alpha):
            def _dirichlet_multinomial_nll(alpha, data_np):
                """
                Compute the negative log-likelihood for a Dirichlet-Multinomial model,
                given 'data_np' (shape M x K) and 'alpha' (shape K, > 0).
                Omits the factorial terms that don't depend on alpha.
                """
                if np.any(alpha <= 0):
                    return np.inf

                alpha_sum = np.sum(alpha)
                nll = 0.0
                for row in data_np:
                    N = np.sum(row)
                    # part that depends on alpha_sum
                    nll -= gammaln(alpha_sum)
                    nll += gammaln(alpha_sum + N)
                    # part that depends on alpha_j
                    for j in range(len(alpha)):
                        nll -= (gammaln(alpha[j] + row[j]) - gammaln(alpha[j]))
                return float(nll)
            return _dirichlet_multinomial_nll(alpha, data_np)

        init_alpha = np.ones(K) + 0.1
        bounds = [(1e-9, None)] * K

        result = minimize(
            fun=objective,
            x0=init_alpha,
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 10000, 'ftol': 1e-5}
        )
        if not result.success:
            print(f"[Warning] MLE not converged: {result.message}")

        return float(result.fun), result.x

    def __LRT_test(self, data1, data2):
        """
        Perform the LRT:
          - Fit combined data => loss_full
          - Fit data1 alone => loss1
          - Fit data2 alone => loss2
          => chi2 = 2 * (loss_full - (loss1 + loss2))
          p-value from Chi2(K).
        """
        total_counts1 = data1.sum(dim=-1).float()
        total_counts2 = data2.sum(dim=-1).float()
        combined_data = torch.cat([data1, data2], dim=0)
        combined_counts = torch.cat([total_counts1, total_counts2], dim=0)

        # Full model
        loss_full, alpha_full = self.__perform_mle(combined_data, combined_counts)
        # data1 alone
        loss1, alpha1 = self.__perform_mle(data1, total_counts1)
        # data2 alone
        loss2, alpha2 = self.__perform_mle(data2, total_counts2)

        chi2_stat = 2.0 * (loss_full - (loss1 + loss2))
        if chi2_stat < 0:
            return float('nan'), float('nan'), alpha_full, alpha1, alpha2

        # Degrees of freedom = K
        K = data1.size(1)
        chi2_val = torch.tensor(chi2_stat, dtype=torch.float)
        p_value = 1.0 - torch.distributions.Chi2(df=K).cdf(chi2_val).item()

        return chi2_stat, p_value, alpha_full, alpha1, alpha2

    def __compare_groups(self, group_1_label, group_2_label, cell_group_column, gene_id):
        group_1 = self[self.obs[cell_group_column] == group_1_label]
        group_2 = self[self.obs[cell_group_column] == group_2_label]

        data1 = torch.tensor(
            group_1[:, group_1.var['geneId'] == gene_id].X.toarray(), dtype=torch.float
        )
        data2 = torch.tensor(
            group_2[:, group_2.var['geneId'] == gene_id].X.toarray(), dtype=torch.float
        )

        total_counts1 = data1.sum(dim=-1)
        total_counts2 = data2.sum(dim=-1)
        non_zero_indices1 = total_counts1 > 9
        non_zero_indices2 = total_counts2 > 9
        data1 = data1[non_zero_indices1]
        data2 = data2[non_zero_indices2]
        total_counts1 = total_counts1[non_zero_indices1].float()
        total_counts2 = total_counts2[non_zero_indices2].float()

        if data1.size(0) == 0 or data2.size(0) == 0:
            return None

        chi2_stat, p_value, alpha_combined, alpha1, alpha2 = self.__LRT_test(data1, data2)
        if not np.isnan(chi2_stat):
            return {
                "chi2_stat": chi2_stat,
                "p_value": p_value,
                "alpha_combined": alpha_combined.tolist(),
                "alpha1": alpha1.tolist(),
                "alpha2": alpha2.tolist()
            }
        return None

    def __filter_genes(self, group_1_label, group_2_label, cell_group_column, min_count=9, min_diff=0.1):
        group_1 = self[self.obs[cell_group_column] == group_1_label]
        group_2 = self[self.obs[cell_group_column] == group_2_label]

        gene_counts_group_1 = np.array(group_1.X.sum(axis=0)).flatten()
        gene_counts_group_2 = np.array(group_2.X.sum(axis=0)).flatten()
        total_gene_counts = gene_counts_group_1 + gene_counts_group_2

        valid_genes = total_gene_counts > min_count
        adata = self[:, valid_genes]

        group_1 = adata[adata.obs[cell_group_column] == group_1_label]
        group_2 = adata[adata.obs[cell_group_column] == group_2_label]

        non_zero_genes_group_1 = np.array((group_1.X != 0).sum(axis=0)).flatten() > min_count
        non_zero_genes_group_2 = np.array((group_2.X != 0).sum(axis=0)).flatten() > min_count
        valid_genes = non_zero_genes_group_1 & non_zero_genes_group_2
        filtered_with_valid_genes = adata[:, valid_genes]
        for gene_id in np.concatenate((group_1.var['geneId'], group_2.var['geneId'])):
            percent_1 = group_1[:, group_1.var['geneId'] == gene_id]
            percent_2 = group_2[:, group_2.var['geneId'] == gene_id]
            percent_1_X = percent_1.X.astype(float)
            percent_2_X = percent_2.X.astype(float)
            percent_1_X = percent_1_X / float(sum(percent_1_X.sum(axis=0)))
            percent_2_X = percent_2_X / float(sum(percent_2_X.sum(axis=0)))
            percent_1.layers['percent'] = percent_1_X
            percent_2.layers['percent'] = percent_2_X
            diff = False
            for transcript_id in np.concatenate((
                group_1[:, group_1.var['geneId'] == gene_id].var['transcriptId'],
                group_2[:, group_2.var['geneId'] == gene_id].var['transcriptId']
            ), axis=0):
                per1 = percent_1[:, percent_1.var['transcriptId'] == transcript_id].layers['percent'].sum(axis=0)
                per2 = percent_2[:, percent_2.var['transcriptId'] == transcript_id].layers['percent'].sum(axis=0)
                diff = abs(per1 - per2) > min_diff
                if diff:
                    break
            if not diff:
                filtered_with_valid_genes = filtered_with_valid_genes[:, filtered_with_valid_genes.var['geneId'] != gene_id]

        return SwitchSearch(filtered_with_valid_genes, self.obs['cell_type'])

    def __find_major_minor_isoforms(self):
        df = self._filtered_anndata.to_df()
        df['cell_type'] = self._filtered_anndata.obs['cell_type']
        df = df.groupby('cell_type').mean().transpose()
        df['geneId'] = self._filtered_anndata.var['geneId']
        df['is_major'] = ''
        for ct in self._filtered_anndata.obs['cell_type'].unique():
            df.loc[df[ct] == df.groupby('geneId')[ct].transform('max'), 'is_major'] += ct + ','
        di = df['is_major'].to_dict()
        d = dict()
        for (k, v) in di.items():
            d[k] = len(v.split(',')) - 1
        tr_ids = list(set(k for (k, v) in d.items() if v == 1))
        return tr_ids

    def find_major_minor_isoforms(self):
        return self.__find_major_minor_isoforms()

    ###############################################################
    # Helper function for Dirichlet-Multinomial Negative Log-Likelihood
    ###############################################################
    def _dirichlet_multinomial_nll(self, alpha, data_np):
        """
        Compute the negative log-likelihood for a Dirichlet-Multinomial model,
        given 'data_np' (shape M x K) and 'alpha' (shape K, > 0).
        Omits the factorial terms that don't depend on alpha.
        """
        if np.any(alpha <= 0):
            return np.inf

        alpha_sum = np.sum(alpha)
        nll = 0.0
        for row in data_np:
            N = np.sum(row)
            # part that depends on alpha_sum
            nll -= gammaln(alpha_sum)
            nll += gammaln(alpha_sum + N)
            # part that depends on alpha_j
            for j in range(len(alpha)):
                nll -= (gammaln(alpha[j] + row[j]) - gammaln(alpha[j]))
        return float(nll)

    def _dm_mle(self, data):
        """
        SciPy-based Dirichlet-Multinomial MLE on a NumPy array.
        data: (M, K)
        Returns: (nll, alpha_hat)
        """
        K = data.shape[1]

        def objective(alpha):
            return self._dirichlet_multinomial_nll(alpha, data)

        init_alpha = np.ones(K) + 0.1
        bounds = [(1e-9, None)] * K

        result = minimize(
            fun=objective,
            x0=init_alpha,
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 10000, 'ftol': 1e-5}
        )
        if not result.success:
            return float('nan'), np.zeros(K)

        return float(result.fun), result.x


    def __fit_single_group(self, group_label, gene_id, cell_group_column='cell_type'):
        """
        Returns (nll, alpha) for a single group's data on a given gene,
        filtered to rows with sum > 9. Uses and updates self._single_fit_cache.
        If no data remains after filtering, returns (np.nan, None).
        """
        cache_key = (group_label, gene_id)
        if cache_key in self._single_fit_cache:
            return self._single_fit_cache[cache_key]

        group_data = self[self.obs[cell_group_column] == group_label]
        mask_gene = (group_data.var['geneId'] == gene_id)
        data = group_data[:, mask_gene].X.toarray()
        total_counts = data.sum(axis=1)
        data = data[total_counts > 9]  # filter

        if data.shape[0] == 0:
            self._single_fit_cache[cache_key] = (np.nan, None)
            return (np.nan, None)

        nll, alpha = self._dm_mle(data)
        self._single_fit_cache[cache_key] = (nll, alpha)
        return (nll, alpha)

    def __fit_combined(self, group_label1, group_label2, gene_id, cell_group_column='cell_type'):
        """
        Returns (nll, alpha) for the combined data from group_label1+group_label2
        on a given gene, filtered. Uses and updates self._combined_fit_cache.
        If no data remains after filtering, returns (np.nan, None).
        """
        cache_key = (group_label1, group_label2, gene_id)
        if cache_key in self._combined_fit_cache:
            return self._combined_fit_cache[cache_key]

        group1 = self[self.obs[cell_group_column] == group_label1]
        mask1 = (group1.var['geneId'] == gene_id)
        data1 = group1[:, mask1].X.toarray()
        total_counts1 = data1.sum(axis=1)
        data1 = data1[total_counts1 > 9]

        group2 = self[self.obs[cell_group_column] == group_label2]
        mask2 = (group2.var['geneId'] == gene_id)
        data2 = group2[:, mask2].X.toarray()
        total_counts2 = data2.sum(axis=1)
        data2 = data2[total_counts2 > 9]

        if data1.shape[0] == 0 or data2.shape[0] == 0:
            self._combined_fit_cache[cache_key] = (np.nan, None)
            return (np.nan, None)

        combined_data = np.vstack([data1, data2])
        nll, alpha = self._dm_mle(combined_data)
        self._combined_fit_cache[cache_key] = (nll, alpha)
        return (nll, alpha)

    def __cached_LRT_test(self, group_label1, group_label2, gene_id, cell_group_column='cell_type'):
        """
        Use the cached single-group fits for group_label1, group_label2
        plus a cached combined fit for (group_label1, group_label2).
        Then do the LRT. Returns (chi2, p_value, alpha_full, alpha1, alpha2) or None.
        """
        loss1, alpha1 = self.__fit_single_group(group_label1, gene_id, cell_group_column)
        loss2, alpha2 = self.__fit_single_group(group_label2, gene_id, cell_group_column)
        if alpha1 is None or alpha2 is None or np.isnan(loss1) or np.isnan(loss2):
            return None

        loss_full, alpha_full = self.__fit_combined(group_label1, group_label2, gene_id, cell_group_column)
        if alpha_full is None or np.isnan(loss_full):
            return None

        chi2_stat = 2.0 * (loss_full - (loss1 + loss2))
        if chi2_stat < 0:
            return None

        # Degrees of freedom = K
        K = len(alpha_full)
        p_value = 1.0 - chi2.cdf(chi2_stat, df=K)
        return chi2_stat, p_value, alpha_full, alpha1, alpha2

    def __compare_groups(self, group_1_label, group_2_label, cell_group_column, gene_id):
        result = self.__cached_LRT_test(group_1_label, group_2_label, gene_id, cell_group_column)
        if result is None:
            return None

        chi2_stat, p_value, alpha_combined, alpha1, alpha2 = result
        transcript_ids = self.var_names[self.var['geneId'] == gene_id].tolist()
        return {
            "chi2_stat": chi2_stat,
            "p_value": p_value,
            "alpha_combined": alpha_combined.tolist(),
            "alpha1": alpha1.tolist(),
            "alpha2": alpha2.tolist(),
            "transcript_ids": transcript_ids
        }

    def __filter_genes(self, group_1_label, group_2_label, cell_group_column, min_count=9, min_diff=0.1):
        group_1 = self[self.obs[cell_group_column] == group_1_label]
        group_2 = self[self.obs[cell_group_column] == group_2_label]

        gene_counts_group_1 = np.array(group_1.X.sum(axis=0)).flatten()
        gene_counts_group_2 = np.array(group_2.X.sum(axis=0)).flatten()
        total_gene_counts = gene_counts_group_1 + gene_counts_group_2

        valid_genes_mask = total_gene_counts > min_count
        adata_slice = self[:, valid_genes_mask]

        group_1_slice = adata_slice[adata_slice.obs[cell_group_column] == group_1_label]
        group_2_slice = adata_slice[adata_slice.obs[cell_group_column] == group_2_label]

        non_zero_genes_group_1 = (group_1_slice.X != 0).sum(axis=0) > min_count
        non_zero_genes_group_2 = (group_2_slice.X != 0).sum(axis=0) > min_count
        final_valid_genes = non_zero_genes_group_1 & non_zero_genes_group_2
        filtered_with_valid_genes = adata_slice[:, final_valid_genes]

        # Additional pass to exclude genes whose transcripts do not differ enough in % usage
        for gene_id in np.unique(filtered_with_valid_genes.var['geneId']):
            sub1 = filtered_with_valid_genes[
                filtered_with_valid_genes.obs[cell_group_column] == group_1_label,
                filtered_with_valid_genes.var['geneId'] == gene_id
            ]
            sub2 = filtered_with_valid_genes[
                filtered_with_valid_genes.obs[cell_group_column] == group_2_label,
                filtered_with_valid_genes.var['geneId'] == gene_id
            ]
            X1 = sub1.X.astype(float).toarray()
            X2 = sub2.X.astype(float).toarray()
            if X1.sum() == 0 or X2.sum() == 0:
                filtered_with_valid_genes = filtered_with_valid_genes[:, filtered_with_valid_genes.var['geneId'] != gene_id]
                continue
            percent_1 = X1 / X1.sum()
            percent_2 = X2 / X2.sum()
            diff_found = False
            for col_idx in range(percent_1.shape[1]):
                if abs(percent_1[:, col_idx].sum() - percent_2[:, col_idx].sum()) > min_diff:
                    diff_found = True
                    break
            if not diff_found:
                filtered_with_valid_genes = filtered_with_valid_genes[:, filtered_with_valid_genes.var['geneId'] != gene_id]

        return AnnDataIso(filtered_with_valid_genes, self.obs['cell_type'])

    def __find_major_minor_isoforms(self):
        df = self.__filtered_anndata.to_df()
        df['cell_type'] = self.__filtered_anndata.obs['cell_type']
        df = df.groupby('cell_type').mean().transpose()
        df['geneId'] = self.__filtered_anndata.var['geneId']
        df['is_major'] = ''
        for ct in self.__filtered_anndata.obs['cell_type'].unique():
            df.loc[df[ct] == df.groupby('geneId')[ct].transform('max'), 'is_major'] += ct + ','
        di = df['is_major'].to_dict()
        d = dict()
        for (k, v) in di.items():
            d[k] = len(v.split(',')) - 1
        tr_ids = list(set(k for (k, v) in d.items() if v == 1))
        return tr_ids

    def find_major_minor_isoforms(self):
        return self.__find_major_minor_isoforms()

    ############################################################
    # New: Wilcoxon-based approach for switching isoforms
    ############################################################
    def __scanpy_wilcoxon_switching_isoforms(self, cell_labels_column='cell_type',
                                             min_fdr=0.05, min_log_fold_change=0.3):
        """
        Mimics the 'get_isoswitches' function you provided, but uses 'self' as the AnnData object.
        Returns a DataFrame of switching isoforms based on Scanpy's rank_genes_groups (Wilcoxon).
        """
        import scanpy as sc

        # We'll work on a copy to avoid mutating 'self'
        adata = self.copy()

        # If the anndata is empty, return empty DataFrame
        if adata.shape[0] == 0:
            return pd.DataFrame()

        # By default, ensure transcriptId is in var
        if 'transcriptId' not in adata.var.columns:
            adata.var['transcriptId'] = adata.var_names

        # Make sure the label column is categorical
        adata.obs[cell_labels_column] = adata.obs[cell_labels_column].astype('category')
        groups = adata.obs[cell_labels_column].cat.categories

        if len(groups) < 2:
            return pd.DataFrame()

        from itertools import combinations
        group_contrasts = list(combinations(groups, 2))
        marker_results_list = {}

        # Perform Wilcoxon-based tests for each pair in both directions
        for group_1, group_2 in group_contrasts:
            try:
                sc.tl.rank_genes_groups(
                    adata, groupby=cell_labels_column, groups=[group_1],
                    reference=group_2, method='wilcoxon', n_genes=adata.shape[0]
                )
                df1 = sc.get.rank_genes_groups_df(adata, group=group_1)
                df1['group_1'] = group_1
                df1['group_2'] = group_2
                df1['contrast'] = f"{group_1}__{group_2}"

                sc.tl.rank_genes_groups(
                    adata, groupby=cell_labels_column, groups=[group_2],
                    reference=group_1, method='wilcoxon', n_genes=adata.shape[0]
                )
                df2 = sc.get.rank_genes_groups_df(adata, group=group_2)
                df2['group_1'] = group_2
                df2['group_2'] = group_1
                df2['contrast'] = f"{group_1}__{group_2}"

                if not df1.empty and not df2.empty:
                    marker_results_list[df1['contrast'].iloc[0]] = pd.concat([df1, df2], ignore_index=True)

            except ValueError:
                pass  # skip if it fails

        if not marker_results_list:
            return pd.DataFrame()

        marker_df = pd.concat(marker_results_list.values(), ignore_index=True)

        # If 'transcriptId' isn't in var, try resetting index
        if 'transcriptId' not in adata.var.columns:
            adata.var.reset_index(inplace=True)

        # Map transcripts to genes
        transcript_to_gene = adata.var.set_index('transcriptId')['geneId'].to_dict()
        marker_df['geneId'] = marker_df['names'].map(transcript_to_gene)

        # Add cell counts
        cell_counts = adata.obs[cell_labels_column].value_counts()
        marker_df['n_cells_group_1'] = marker_df['group_1'].map(cell_counts)
        marker_df['n_cells_group_2'] = marker_df['group_2'].map(cell_counts)
        marker_df['total_cells'] = marker_df['n_cells_group_1'] + marker_df['n_cells_group_2']

        # Adjust p-values and filter
        marker_df['adj_pval'] = multipletests(marker_df['pvals_adj'], method='fdr_bh')[1]
        marker_df_filtered = marker_df[
            (marker_df['adj_pval'] <= min_fdr) &
            (marker_df['logfoldchanges'].abs() >= min_log_fold_change)
        ]

        def assign_direction(df):
            return df.assign(
                direction=np.where(
                    df['group_1'] == df['contrast'].str.split("__").str[0],
                    df['logfoldchanges'],
                    -df['logfoldchanges']
                )
            )

        isoswitch_df = (
            marker_df_filtered
            .groupby(['geneId', 'contrast'])
            .apply(assign_direction)
            .reset_index(drop=True)
            .groupby(['geneId', 'contrast'])
            .filter(lambda x: (
                len(x['group_1'].unique()) > 1 and
                len(x['names'].unique()) > 1 and
                x['direction'].abs().sum() != 0
            ))
        )

        def calculate_percent_expression(adata, groupby_column, group, transcript):
            subset = adata[adata.obs[groupby_column] == group, transcript]
            return (subset.X > 0).mean() * 100

        percent_expressions = isoswitch_df.apply(
            lambda row: pd.Series({
                'percent_expressed_group_1': calculate_percent_expression(adata, cell_labels_column, row['group_1'], row['names']),
                'percent_expressed_group_2': calculate_percent_expression(adata, cell_labels_column, row['group_2'], row['names'])
            }),
            axis=1
        )
        isoswitch_df = pd.concat([isoswitch_df, percent_expressions], axis=1)
        isoswitch_df.sort_values(by='adj_pval', inplace=True)

        return isoswitch_df

    ################################################################
    # Updated find_switching_isoforms(...) with method switch
    ################################################################
    def find_switching_isoforms(
        self,
        cell_group_column='cell_type',
        min_count=30,
        min_diff=0.2,
        method="ScanpyWilcoxon",
        min_fdr=0.05,
        min_log_fold_change=0.5
    ):
        """
        Finds switching isoforms. By default uses the Wilcoxon-based method
        (Scanpy's rank_genes_groups). If you want the original Dirichlet-based
        approach, call with method='Dirichlet'.

        For method='Dirichlet', 'min_count' and 'min_diff' apply.
        For method='ScanpyWilcoxon', 'min_fdr' and 'min_log_fold_change' apply.
        """
        if method == "Dirichlet":
            # Original approach
            cell_types = self.obs[cell_group_column].unique()
            gene_ids = self.var['geneId'].unique()

            def process_gene(obj, group_1_label, group_2_label, gene_id):
                result = obj.__compare_groups(group_1_label, group_2_label, cell_group_column, gene_id)
                if result:
                    return {
                        'gene_id': gene_id,
                        'group_1': group_1_label,
                        'group_2': group_2_label,
                        'p_value': result['p_value'],
                        'chi2_stat': result['chi2_stat'],
                        'alpha_combined': result['alpha_combined'],
                        'alpha1': result['alpha1'],
                        'alpha2': result['alpha2'],
                        'transcript_ids': result['transcript_ids']
                    }
                return None

            results = []
            for group_1_label, group_2_label in combinations(cell_types, 2):
                filtered_adata = self.__filter_genes(group_1_label, group_2_label, cell_group_column, min_count, min_diff)
                pairwise_results = []
                for gene_id in gene_ids:
                    if gene_id not in filtered_adata.var['geneId'].values:
                        continue
                    res = process_gene(filtered_adata, group_1_label, group_2_label, gene_id)
                    if res is not None:
                        pairwise_results.append(res)
                results.extend(pairwise_results)

            results_df = pd.DataFrame(results)
            self.relevant_genes = results_df
            return results_df
        else:
            # New default approach: Wilcoxon-based
            isoswitch_df = self.__scanpy_wilcoxon_switching_isoforms(
                cell_labels_column=cell_group_column,
                min_fdr=min_fdr,
                min_log_fold_change=min_log_fold_change
            )
            return isoswitch_df

    def plot_relevant_genes(self, gene_id, p_value_threashold=0.05, data=None):
        if data is not None:
            self.relevant_genes = data
        if not hasattr(self, 'relevant_genes') or self.relevant_genes is None:
            raise Exception('Please run the `%s` method on this object in order to preprocess the needed data.' % ('find_switching_isoforms'))
        switching_genes = self.relevant_genes[
            (self.relevant_genes['gene_id'] == gene_id)
            & (self.relevant_genes['p_value'] < p_value_threashold)
        ]
        switching_genes[['cell_type']] = self.obs[['cell_type']]
        column_names = self.to_df().columns.tolist()
        transcr_indexes = list(map(str, column_names))

        cell_types_compare = switching_genes[
            (self.relevant_genes['gene_id'] == gene_id)
            & (switching_genes['p_value'] < p_value_threashold)
        ][['group_1', 'group_2']].values.tolist()
        cell_types_compare = list(set([x for xs in cell_types_compare for x in xs]))
        tr_ids = switching_genes[
            (self.relevant_genes['gene_id'] == gene_id)
            & (switching_genes['p_value'] < p_value_threashold)
        ]['transcript_ids'].values.tolist()
        tr_ids_json = [tr_id for tr_id in tr_ids]
        tr_ids_flattened = list(set([x for xs in tr_ids_json for x in xs]))
        fig = plt.figure()

        data = self.to_df()
        data[['cell_type']] = self.obs[['cell_type']]
        genes_sum = data[data['cell_type'].isin(cell_types_compare)][tr_ids_flattened + ['cell_type']].groupby('cell_type').sum()
        genes_nonzero = data[data['cell_type'].isin(cell_types_compare)].groupby('cell_type').apply(lambda x: (x != 0).sum())[tr_ids_flattened]

        color_data = genes_nonzero[tr_ids_flattened].to_numpy()
        size_data = genes_sum[tr_ids_flattened].to_numpy()
        x_labels = tr_ids_flattened

        # 'scattermap' presumably from your code
        ax = scattermap(
            color_data, marker_size=size_data, square=True, cmap="Reds",
            cbar_kws={"label": "Non-zeros"},
            xticklabels=x_labels, yticklabels=cell_types_compare
        )
        mk_size = 60
        ax.scatter(-1, -1, label=f"{np.amax(size_data):0.1f}", marker="o", c="r", s=mk_size)
        ax.scatter(-1, -1, label=f"{np.mean(size_data):0.1f}", marker="o", c="r", s=mk_size * 0.5)
        if np.any(size_data > 0):
            nz_min = np.amin(size_data[np.nonzero(size_data)])
        else:
            nz_min = 0.0
        ax.scatter(-1, -1, label=f"{nz_min:0.1f}", marker="o", c="r", s=mk_size * 0.1)
        ax.legend(loc="upper left", bbox_to_anchor=(1.27, -0.05))
        fig.tight_layout()

    def plot_relevant_genes(self, gene_id, p_value_threashold=0.05, data=None):
        if data is not None:
            self.relevant_genes = data
        if not hasattr(self, 'relevant_genes') or self.relevant_genes is None:
            raise Exception('Please run the `%s` method on this object in order to preprocess the needed data.' % ('find_switching_isoforms'))
        switching_genes = self.relevant_genes[
            (self.relevant_genes['gene_id'] == gene_id)
            & (self.relevant_genes['p_value'] < p_value_threashold)
        ]
        switching_genes[['cell_type']] = self.obs[['cell_type']]
        column_names = self.to_df().columns.tolist()
        transcr_indexes = list(map(str, column_names))

        cell_types_compare = switching_genes[
            (self.relevant_genes['gene_id'] == gene_id)
            & (switching_genes['p_value'] < p_value_threashold)
        ][['group_1', 'group_2']].values.tolist()
        cell_types_compare = list(set([x for xs in cell_types_compare for x in xs]))
        tr_ids = switching_genes[
            (self.relevant_genes['gene_id'] == gene_id)
            & (switching_genes['p_value'] < p_value_threashold)
        ]['transcript_ids'].values.tolist()
        tr_ids_json = [tr_id for tr_id in tr_ids]
        tr_ids_flattened = list(set([x for xs in tr_ids_json for x in xs]))
        fig = plt.figure()

        data = self.to_df()
        data[['cell_type']] = self.obs[['cell_type']]
        genes_sum = data[data['cell_type'].isin(cell_types_compare)][tr_ids_flattened + ['cell_type']].groupby('cell_type').sum()
        genes_nonzero = data[data['cell_type'].isin(cell_types_compare)].groupby('cell_type').apply(lambda x: (x != 0).sum())[tr_ids_flattened]

        # 'scattermap' is presumably from your code
        color_data = genes_nonzero[tr_ids_flattened].to_numpy()
        size_data = genes_sum[tr_ids_flattened].to_numpy()
        x_labels = tr_ids_flattened
        ax = scattermap(
            color_data, marker_size=size_data, square=True, cmap="Reds",
            cbar_kws={"label": "Non-zeros"},
            xticklabels=x_labels, yticklabels=cell_types_compare
        )
        mk_size = 60
        ax.scatter(-1, -1, label=f"{np.amax(size_data):0.1f}", marker="o", c="r", s=mk_size)
        ax.scatter(-1, -1, label=f"{np.mean(size_data):0.1f}", marker="o", c="r", s=mk_size * 0.5)
        if np.any(size_data > 0):
            nz_min = np.amin(size_data[np.nonzero(size_data)])
        else:
            nz_min = 0.0
        ax.scatter(-1, -1, label=f"{nz_min:0.1f}", marker="o", c="r", s=mk_size * 0.1)
        ax.legend(loc="upper left", bbox_to_anchor=(1.27, -0.05))
        fig.tight_layout()

In [ ]:
# | export
def create_iso_adata(
    path: str  # Path to a tab-separated file, typically the Sicelore output with transcript counts.
):
    """
    Creates an AnnData object containing isoform counts from a Sicelore output file.

    Parameters:
    - path (str): Path to the input file, which is expected to be tab-separated and contain:
        - `transcriptId`: Transcript identifiers.
        - `geneId`: Gene identifiers.
        - Transcript counts for individual barcodes (cells) as additional columns.

    Returns:
    - adata_iso (AnnData): An AnnData object with isoform counts.
    """

    # Open the input file and read it into a Pandas DataFrame
    with open(path) as isoforms:
        df = pd.read_table(isoforms, sep='\t')

    # Extract the count matrix
    # - `df.iloc[0:, 2:]`: assuming these are counts.
    counts = df.iloc[0:, 2:]

    # Create an AnnData object with the count matrix

    adata_iso = ad.AnnData(counts).transpose()

    # Assign transcript IDs as variable names (columns in the original file)
    adata_iso.var_names = df['transcriptId'].to_list()

    # Add additional variable-level metadata
    # - Include both `transcriptId` and `geneId` as annotations for the variables.
    adata_iso.var = df[['transcriptId', 'geneId']]

    # Create observation-level metadata for barcodes (cells)
    # - Extract column names (barcodes) starting from the 3rd column (index 2).
    barcodes = {'barcodes': df.columns.values[2:]}
    barcodes = pd.DataFrame(data=barcodes)

    # Assign the barcodes DataFrame to the AnnData object as observation metadata
    adata_iso.obs = barcodes

    # Set observation names (row indices) in the AnnData object to match the barcodes
    adata_iso.obs_names = barcodes['barcodes'].tolist()

    # Return the AnnData object containing isoform counts
    return adata_iso